# Project - Sentiment Analysis Using Deep Learning

<img src="banner.jpeg">

When it comes to text data, sentiment analysis is one of the most widely performed analysis on it. Sentiment Analysis has been through tremendous improvements from the days of classic methods to recent times where in the state of the art models utilize deep learning to improve the performance.

Convolutional Neural Networks or CNNs are the work-horse of the deep learning world. They have, in some sense, brought deep learning research into mainstream discussions. The advancements in the image classification world has left even humans behind.

<img src="cnn.png">
In this project, we will attempt at performing sentiment analysis utilizing the power of CNNs.

In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

Using TensorFlow backend.


## Load Movie Review Dataset

In [0]:
dataset = pd.read_csv(r'movie_reviews.csv')

In [3]:
# take a peek at the data
print(dataset.head())
reviews = np.array(dataset['review'])
sentiments = np.array(dataset['sentiment'])

                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


### Prepare Train Test Split

In [0]:
# build train and test datasets
train_reviews = reviews[:35000]
train_sentiments = sentiments[:35000]
test_reviews = reviews[35000:]
test_sentiments = sentiments[35000:]

## Preprocessing

To prepare text data for our deep learning model, we transform each review into a sequence.
Every word in the review is mapped to an integer index and thus the sentence turns into a sequence of numbers.

To perform this transformation, keras provides the ```Tokenizer```

In [0]:
t = Tokenizer()
# fit the tokenizer on the documents
t.fit_on_texts(train_reviews)

In [0]:
train_sequences = t.texts_to_sequences(train_reviews)

In [0]:
test_sequences = t.texts_to_sequences(test_reviews)

In [8]:
print("Vocabulary size={}".format(len(t.word_counts)))
print("Number of Documents={}".format(t.document_count))

Vocabulary size=105664
Number of Documents=35000


### Sequence Normalization

Not all reviews are of same length. To handle this difference in length of reviews, we define a maximum length.
For reviews which are smaller than this length, we padd them with zeros which longer ones are truncated

In [0]:
MAX_SEQUENCE_LENGTH = 500

In [0]:
# pad dataset to a maximum review length in words
X_train = sequence.pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
X_test = sequence.pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

### Encoding Labels

The dataset contains labels of the form positive/negative. The following step encodes the labels using ```sklearn's``` ```LabelEncoder```

In [0]:
le = LabelEncoder()
num_classes=2 # positive -> 1, negative -> 0

In [0]:
y_train = le.fit_transform(train_sentiments)
y_test = le.transform(test_sentiments)

In [0]:
VOCAB_SIZE = len(t.word_counts)+1

## Prepare the Model

Since textual data is a sequence of words, we utilize ```1D``` convolutions to scan through the sentences.
The model first transforms each word into lower dimensional embedding/vector space followed by 1d convolutions and then passing the data through dense layers before the final layer for classification

In [0]:
EMBED_SIZE = 32
EPOCHS=2
BATCH_SIZE=128

In [15]:
# create the model
model = Sequential()
model.add(Embedding(VOCAB_SIZE, EMBED_SIZE, input_length=MAX_SEQUENCE_LENGTH))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           3381280   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 500, 32)           3104      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 250, 32)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 8000)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 250)               2000250   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 251       
Total params: 5,384,885
Trainable params: 5,384,885
Non-trainable params: 0
_________________________________________________________________


## Model Training

In [16]:
# Fit the model
model.fit(X_train, y_train, 
          validation_split=0.2,
          epochs=EPOCHS, 
          batch_size=BATCH_SIZE, 
          verbose=1)

Train on 28000 samples, validate on 7000 samples
Epoch 1/2
28000/28000 [==============================] - 45s 2ms/step - loss: 0.4164 - acc: 0.7824 - val_loss: 0.2577 - val_acc: 0.8954
Epoch 2/2
28000/28000 [==============================] - 44s 2ms/step - loss: 0.1369 - acc: 0.9504 - val_loss: 0.2709 - val_acc: 0.8971


## Model Evaluation

In [17]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 90.13%
